In [1]:
"""
Author: Trenz Electronic GmbH / Kilian Jahn 27.09.2019 - last edited 06.10.2020
Edited within the Python version:   3.8.3 / 3.7.3

This notebook is an example on using the ADC of the Trenz modules TEI0015,
TEI0016 a/b and TEI023 a/b. 
It is compatible to 'Jupyter' (min. 5.7.8) and 'Jupyter-Lab' (min 2.1.5)

The notebook features data acquisition, conversion to standard integer
and floating point values, and Fourier transformation.
Further explonations are available in the Trenz Electronic wiki for the above
mentioned modules:
https://wiki.trenz-electronic.de/display/PD/TEI0015+-+Jupyter+Demo
https://wiki.trenz-electronic.de/display/PD/TEI0016+-+Jupyter+Demo
https://wiki.trenz-electronic.de/display/PD/TEI0023+-+Jupyter+Demo
"""



import sys                                # Reading installed python Version
import copy                               # Deep coping of module parameters

import serial                             # Serial/comport connection
import serial.tools.list_ports            # Reading available comport

%matplotlib widget
from matplotlib import pyplot as plt      # Interactive Graphs
import numpy as np                        # Change values of collected data easily

import ipywidgets as widgets              # GUI
from IPython.display import clear_output  # Clearing the GUI / plots

import MOD_TEI00xyCodeModule                  # Functions for collecting and analysing ADC data

from datetime import datetime
from datetime import timedelta


# Global variables---------------------------------------------------------------------------------

# ############## Set default values for comport here #########################
#
# No default comport = 'COMx' -> To set a default comport 4, set to 'COM4'
defaultPort = 'COMx'
# 
#
# ############################################################################


# Select the graph style of the FFT by 
# reciprocal commenting and uncommenting both lines
fftPlot = 'logarithmic'
#fftPlot = 'linear'


# Module parameters
TEI0015 = {
    'ID': 1, 'module': 'TEI0015 - 2 MS', 'featuresSpanCom&HighZ': True,
    'AdcSamplingRate' : 2000000, 'voltageRef': 10, 'AdcResolution': 18, 'AdcTreshold' : 131052,
    'gain 1': 1, 'gain 2': 2, 'gain 4': 4, 'gain 8': 8,
    'gainList' : [1, 2, 4, 8]
}
TEI0016A = {
    'ID': 2, 'module': 'TEI0016A - 0.5 MS','featuresSpanCom&HighZ': False,
    'AdcSamplingRate': 500000, 'voltageRef': 10, 'AdcResolution': 16, 'AdcTreshold': 32763,
    'gain 1': 1, 'gain 2': 2, 'gain 4': 4, 'gain 8': 8,
    'gainList': [1, 2, 4, 8]
}
TEI0016B = {
    'ID': 3, 'module': 'TEI0016B - 1 MS', 'featuresSpanCom&HighZ': False,
    'AdcSamplingRate': 1000000, 'voltageRef': 10, 'AdcResolution': 16, 'AdcTreshold': 32763,
    'gain 1': 1, 'gain 2': 2, 'gain 4': 4, 'gain 8': 8,
    'gainList': [1, 2, 4, 8]
}
TEI0023A = {
    'ID': 4, 'module': 'TEI0023A - 2 MS', 'featuresSpanCom&HighZ': True,
    'AdcSamplingRate': 2000000, 'voltageRef': 11, 'AdcResolution': 18, 'AdcTreshold': 131052,
    'gain 0.25': 1, 'gain 0.5': 2, 'gain 1': 3, 'gain 2': 4, 'gain 4': 5, 'gain 8': 6, 'gain 16': 7,
    'gainList': [0.25, 0.5, 1, 2, 4, 8, 16]
}
TEI0023B = {
    'ID': 5, 'module': 'TEI0023B - 1.8 MS', 'featuresSpanCom&HighZ': True,
    'AdcSamplingRate': 1800000, 'voltageRef': 11, 'AdcResolution': 20, 'AdcTreshold': 524287,
    'gain 0.25': 1, 'gain 0.5': 2, 'gain 1': 3, 'gain 2': 4, 'gain 4': 5, 'gain 8': 6, 'gain 16': 7,
    'gainList': [0.25, 0.5, 1, 2, 4, 8, 16]
}
# The module, to which the Notebook is connected to, 
# is stored together with its settings as the first element
modules = ['connectedModule', TEI0015, TEI0016A, TEI0016B, TEI0023A, TEI0023B]



# GUI function ------------------------------------------------------------------------------------
    
# Setup notebook and GUI to a module
def selectComport(selection):    
    global handleGain
    
    if selection != 'Select comport':                       # No action for first entry  
        # Cut from selection the comport
        for index, character in enumerate(selection):
            if character == ' ':
                comport = str(selection[:int(index)])
                break
        
        msg = moduleParameters('setup', comport)
        
        # Uncheck module features from a previous connection.
        handleSquareWave.value = False  
        handleSpanCom.value = False
        handleHighZ.value = False
        handleStartDataAcq.value = False
        
        if msg == 'Connected to ':                          # Initialize to the selected comport
            print("test")
            disableGui = False
            if defaultPort == 'COMx':
                msg = msg + 'port ' + comport + ' / ' + moduleParameters('module')
            else:                                           # Initialize to the chosen default comport
                msg = msg + 'default port ' + defaultPort + ' / ' + moduleParameters('module')
            textBoxAdd(msg)
            
            # Update gain slider and apply default gain of one
            handleGain.options = moduleParameters('gainList')  # List updating calls its handler function.
            handleGain.value = 2                            # Call synchs handler function for next call.
            modules[0]['suppressExecution'] = True          # Allow execution of handler function.
            handleGain.value = 1
            #performDataAcquisitionFFT()
        else:                                               # msg = 'Not a module on port '
            disableGui = True
            textBoxAdd(msg + comport)
        
        # Enable or disable the GUI
        handleGain.disabled = disableGui
        handleSquareWave.disabled = disableGui
        handleSamples.disabled = disableGui
        #handleButton.disabled = disableGui
        handleStartDataAcq.disabled = disableGui
        #handleStopDataAcq.disabled = disableGui
        # En- or dis- able module specific features. 
        # (not True := if modules[0]... = True and disableGui = False)
        handleSpanCom.disabled = not (moduleParameters('featuresSpanCom&HighZ') and not disableGui)
        handleHighZ.disabled = not (moduleParameters('featuresSpanCom&HighZ') and not disableGui)
        #handleSpanCom.disabled = not (moduleParameters('featuresSpanCom&HighZ&StartDataAcq&StopDataAcq') and not disableGui)
        #handleHighZ.disabled = not (moduleParameters('featuresSpanCom&HighZ&StartDataAcq&StopDataAcq') and not disableGui)
        #handleStartDataAcq.disabled = not (moduleParameters('featuresSpanCom&HighZ&StartDataAcq&StopDataAcq') and not disableGui)
        #handleStartDataAcq.disabled = not (moduleParameters('featuresSpanCom&HighZ&StartDataAcq&StopDataAcq') and not disableGui)


def sliderGain(gain):
    # Altering the slider list elements during 'setup' calls this function. 'suppressExecution' 
    # prevents execution.
    if moduleParameters('suppressExecution'):
        text = moduleParameters('updateGain', gain)
        textBoxAdd(text)

    
def squareWaveSignal(marker):
    text = moduleParameters('squareWaveGen')
    textBoxAdd(text)

    
def startDataAcq(marker):
    text = moduleParameters('startDataAcq')
    textBoxAdd(text)
    

def sliderSamples(samples):
    text = moduleParameters('updateSamples', samples)
    textBoxAdd(text)
    
    
def spanCompression(marker):
    text = moduleParameters('spanCompression')
    textBoxAdd(text)

    
def highZMode(marker):
    text = moduleParameters('highZMode')
    textBoxAdd(text)
    

def textBoxAdd(text):
    global handleMessageBox
    handleMessageBox.value = str(text) + "\n" + handleMessageBox.value
    



# Data related functions --------------------------------------------------------------------------

# Setup notebook to a module, set & get parameters of the module.
def moduleParameters(key, optionalArgument = None):
    global modules, defaultPort, handleSquareWave, handleSpanCom, handleHighZ, handleStartDataAcq
    
    # Connecting to a module, store the connected module and its settings as fifth element in modules[]
    if key == 'setup': 
        idTemp = MOD_TEI00xyCodeModule.getModuleId(optionalArgument)
        if 0 < idTemp < 6:
            modulesTemp = copy.deepcopy(modules[idTemp])
            modulesTemp.update({'comport': optionalArgument, 'suppressExecution': False ,
                'gainCommand': 0, 'gainValue': 0, 'samplesActual': 16 , 
                'statusGen': 'f', 'statusSpanCom': 's', 'statusHighZ': 'h', 'statusStartData': 'd'})
            modules[0] = modulesTemp
            return 'Connected to '
        else:
            return 'Not a module on port '        
    elif key == 'updateGain':           # Gain was changed
        try:
            opaGain = float(optionalArgument)
            # Define plot limits acording to the gain. Apply hardware input limit of 10 V
            inputRange = modules[0]['voltageRef'] / opaGain
            if inputRange >= 10:
                inputRange = 10
            temp = modules[0]['gain ' + str(optionalArgument)]
            MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], str(temp))
            modules[0]['gainValue'] = optionalArgument
            modules[0]['gainCommand'] = temp
            modules[0]['plotScale'] = inputRange
            return 'Set ADC gain to ' + str(opaGain) + ', measurement range is +/- ' + str(inputRange) + ' V'
        except:
            return 'Error setting gain'
    elif key == 'squareWaveGen':        # Status of the square wave generator was changed
        try:            
            if handleSquareWave.value:
                modules[0]['statusGen'] = 'F'                
                text = 'Square wave generator ON [10 kHz, +/- 3.3 V]'
            else:
                modules[0]['statusGen'] = 'f'
                text = 'Square wave generator OFF'
            MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusGen'])
        except:
            text = 'Error, changing state square wave signal'
        return text    
    elif key == 'spanCompression':      # Status of the square wave generator was changed
        try:
            if handleSpanCom.value:
                modules[0]['statusSpanCom'] = 'S'
                text = 'Input Span Compression enabled'
            else:
                modules[0]['statusSpanCom'] = 's'
                text = 'Input Span Compression disabled'
            MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusSpanCom'])
            modules[0]['statusSpanCompression'] = optionalArgument # Save status
        except:
            text = 'Error, changing state Input Span Compression'
        return text    
    elif key == 'highZMode':            # Status of the square wave generator was changed
        try:
            if handleHighZ.value:
                modules[0]['statusHighZ'] = 'H'
                text = 'High-Z Mode enabled'
            else:
                modules[0]['statusHighZ'] = 'h'
                text = 'High-Z Mode disabled'
            MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusHighZ'])
            modules[0]['statusHighZ'] = optionalArgument # Save status
        except:
            text = 'Error, changing state High-Z Mode'
        return text   
    elif key == 'startDataAcq':         # Status of the Start Data Acquisition button was changed
        try:
            if handleStartDataAcq.value:
                modules[0]['statusStartData'] = 'D'
                text = 'Start Data Acquisition ON'
                performDataAcquisitionFFT()
            else:
                modules[0]['statusStartData'] = 'd'
                text = 'Start Data Acquistion OFF'
            MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusStartData'])
            modules[0]['statusStartData'] = optionalArgument # Save status
        except:
            text = 'Error, changing state Start Data Acquisition'
        return text   
    elif key == 'updateSamples':        # Length of samples was changed
        modules[0]['samplesActual'] = int(optionalArgument) # Save value
        return 'Sample length set to ' + str(optionalArgument) + ' kS'    
    elif key == 'reapplyStoredSetting': # Reapply settings before DataAcquisition / FFT
        MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['gainCommand'])
        MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusGen'])
        MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusSpanCom'])
        MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusHighZ'])
        MOD_TEI00xyCodeModule.sendCommand(modules[0]['comport'], modules[0]['statusStartData'])
        return None        
    else:
        if optionalArgument == None:
            temp = modules[0][key]
        else:
            temp = modules[optionalArgument][key]
        return temp

    
# This function contains all the code for data collection from the module
def performDataAcquisitionFFT():
    global modules, defaultPort, handleSquareWave, handleSpanCom, handleHighZ, handleStartDataAcq
    try:
        # Initialize variables
        signalVoltList = []
        signalFloatList = []
        signalIntList = []
        endtime = datetime.now()+timedelta(days=730) # end time is 2 years from the start time
        curtime = datetime.now() # gets the current time
        i = 1
        
        while curtime<=endtime:
            if handleStartDataAcq.value==False:
                break
            # If the module lost its gain and generator setting
            moduleParameters('reapplyStoredSetting')
            # Determine gain parameter
            if handleSpanCom.value:
                gain = float(moduleParameters('gainValue') / 0.8)
            else:
                gain = moduleParameters('gainValue')

            # Collect adc data and convert to voltage
            signalVolt, signalFloatNormalized, signalInteger = MOD_TEI00xyCodeModule.dataCollect(
                moduleParameters('comport'), moduleParameters('samplesActual'), moduleParameters('ID'), gain)
            
            # Store the adc data
            signalVoltList.append(signalVolt)
            signalFloatList.append(signalFloatNormalized)
            signalIntList.append(signalInteger)
            if i%10 == 0: # stores the data in groups of 10
                name = str(datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")) # gets the current date and time
                with open(name, 'w') as f: # names the file after the date and time
                    f.write("signalVolt: " + str(signalVoltList) + "\n") 
                    signalVoltList = []
                    #f.write("signalFloatNormalized: " + str(signalFloatList) + "\n")
                    #signalFloatList = []
                    #f.write("signalInteger: " + str(signalIntList) + "\n")
                    #signalIntList = []
                    f.close()
                    print("Data stored in file: " + name)
            i += 1
            curtime = datetime.now() # Updates the current time
            
        print("Data Acquisition Done")
        return
    except:
        textBoxAdd('Runtime error performing adc data collection')


# Plot the graphs, labels and set its boundaries
def plottingGraphs(plotFigure, labelPlot, labelAxisX, labelAxisY, plotData):
    plt.figure(plotFigure)
    plt.clf()  # Clear figure 
    plt.cla()  # and axes        
    
    if plotFigure == 'Wave':
        signalVolt, signalInteger = plotData
        # Process data for plotting, convert from samples to Milli Seconds
        adcSignalslength = len(signalVolt)
        periode = 1000 * 1 / moduleParameters('AdcSamplingRate')
        duration = np.arange(0, adcSignalslength * periode, periode)
        
        scale = float(moduleParameters('plotScale'))
        axisLimits = [0, duration[-1], -scale, scale]
        plotData = [duration ,signalVolt]   
        
        adcLimitsExceeded = MOD_TEI00xyCodeModule.signalLimitsExceed(signalInteger, 
            moduleParameters('AdcTreshold'))        
        if adcLimitsExceeded > 10:
            text = 'WARNING - ' + str(adcLimitsExceeded) + ' samples exceed measurement range'
            handleAxes = plt.gca()
            handleAxes.text(0.5, 0.5, text, color = 'red', backgroundcolor='white',
                     transform=handleAxes.transAxes, horizontalalignment='center')
            textBoxAdd('WARNING - INPUTSIGNAL EXCEDDS MEASURMENT LIMITS !')
    
    elif plotFigure == 'FFT' and fftPlot == 'logarithmic':
        frequencies, level = plotData        
        labelAxisX = 'Frequency / $Hz$'
        # Process data for plotting, find zoom edges  
        maxFreq = moduleParameters('AdcSamplingRate') / 2
        minAmpli = int(min(level) * 0.87)
        
        plotData = [frequencies, level]
        axisLimits = [1, maxFreq + 5, minAmpli, 0]        
        
        plt.gca().set_xscale('Log')
        plt.gca().grid(b=True, which='major', color='gray', linestyle='-')
        plt.gca().grid(b=True, which='minor', color='darkgray', linestyle='--')

    elif plotFigure == 'FFT' and fftPlot == 'linear':
        frequencies, level = plotData        
        # Process data for plotting, convert to Kilo Hertz, find zoom edges                      
        frequencies = [y / 1000 for y in frequencies]   
        maxFreq = moduleParameters('AdcSamplingRate') / (2 * 1000)
        minAmpli = int(min(level) * 0.87)
        
        plotData = [frequencies, level]
        axisLimits = [0.0001, maxFreq + 5, minAmpli, 0]
    
    plt.gca().grid(True)  
    plt.suptitle(labelPlot, fontsize='13', fontweight='bold')
    plt.ylabel(labelAxisY)
    plt.xlabel(labelAxisX)
    plt.plot(plotData[0], plotData[1])   
    plt.ylim(axisLimits[2],axisLimits[3])
    plt.xlim(axisLimits[0],axisLimits[1])
    plt.show()
    
    

# Define GUI---------------------------------------------------------------------------------------

handleMessageBox = widgets.Textarea(layout=widgets.Layout(width='50%', height='100%'))
handleSerial = widgets.Dropdown(description = 'Comport')
handleGain = widgets.SelectionSlider(description='Gain', options=['1', '2', '4', '8'], disabled = True)
handleSquareWave = widgets.Checkbox(value=False, description='Activate square wave signal', disabled = True)
handleSamples = widgets.SelectionSlider(description='Samples kS', 
                        options=['16', '32', '64', '128', '256', '512', '1024'], disabled = True)
handleSpanCom = widgets.Checkbox(value=False, description='Input Span Compression', disabled = True)
handleHighZ = widgets.Checkbox(value=False, description='High-Z mode', disabled = True)
handleStartDataAcq = widgets.Checkbox(value=False, description='Start Data Acquisition', disabled = True)

#while handleStartDataAcq.value == True:
#    performDataAcquisitionFFT(handleStartDataAcq)
    
#handleButton = widgets.Button(description='Capture FFT data', disabled = True)
#handleButton.on_click(performDataAcquisitionFFT)

# Bind GUI elements to their functions
widgets.interactive(selectComport, selection = handleSerial)
widgets.interactive(sliderGain, gain = handleGain)
widgets.interactive(squareWaveSignal, marker = handleSquareWave)
widgets.interactive(sliderSamples, samples = handleSamples)
widgets.interactive(spanCompression, marker = handleSpanCom)
widgets.interactive(highZMode, marker = handleHighZ)
widgets.interactive(startDataAcq, marker = handleStartDataAcq)

# Show the GUI
handleBox = widgets.HBox([handleGain, handleSquareWave, handleStartDataAcq])
handleBoxFeatures = widgets.HBox([handleSpanCom, handleHighZ])
display(handleMessageBox, handleSerial, handleBox, handleSamples, handleBoxFeatures)
#display(handleMessageBox, handleSerial, handleBox, handleSamples, handleBoxFeatures, handleButton)



# Programm initialisation -------------------------------------------------------------------------

# Show python version
versionFound = sys.version 
for index, character in enumerate(versionFound):        # Cut string to version number 
    if character == ' ':
        versionFound = str(versionFound[:int(index)])
        break
textBoxAdd('Python version ' + versionFound + ' found' 
    + ' / minimum version 3.7.3 required')

# Gather all comports and fill them into the dropdown / serial list
optionList = ['Select comport']
ports = serial.tools.list_ports.comports()
dropListDefaultPortIndex = None
for port, desc, hwid in sorted(ports):
    if desc[:15] == 'USB Serial Port':                  # Modules desc beginns with this
        temp = MOD_TEI00xyCodeModule.getModuleId(port)
        if 0 < temp < 6:
            desc = moduleParameters('module', temp)     # Replace OS desc with modules name
            if port == defaultPort:                     # If default comport found, store its index
                dropListDefaultPortIndex = len(optionList)
    temp = port + " \t" + desc + " \t" + hwid
    optionList.append(temp)
handleSerial.options = optionList

# Initialize the Notebooks GUI, before the comports have been listed and "filtered", ... .
if dropListDefaultPortIndex == None and defaultPort != 'COMx':    # Wrong / faulty default comport
    defaultPort = 'COMx'
    textBoxAdd('Select comport / module\nNot a module on default comport')
elif dropListDefaultPortIndex:                          # Existing default comport
    handleSerial.index = dropListDefaultPortIndex
    defaultPort = 'COMx'
else:                                                   # No default comport
    textBoxAdd('Select comport / module')

    

Textarea(value='', layout=Layout(height='100%', width='50%'))

Dropdown(description='Comport', options=(), value=None)

SelectionSlider(description='Samples kS', disabled=True, options=('16', '32', '64', '128', '256', '512', '1024…

Error determine module ID
